In [2]:
import torch
import torch.nn as nn


In [22]:
# scores shape is [batch_size, seq_len,num_experts]
import torch.nn.functional as F
batch_size = 2
seq_len = 4
num_experts = 3
scores = torch.randn( (batch_size, seq_len, num_experts))
top_k_scores, top_k_indices = scores.topk(k=2, dim=-1)
print(top_k_scores.shape)
mask = torch.zeros_like(scores).scatter_(
            dim=-1, index=top_k_indices, value=1.0)    
print(mask.shape)
masked_scores = scores * mask
# for 0-> -inf 
masked_scores = masked_scores.masked_fill(mask == 0, float('-inf'))
print(masked_scores)

# softmax
masked_scores = F.softmax(masked_scores, dim=-1)
print(masked_scores)



torch.Size([2, 4, 2])
torch.Size([2, 4, 3])
tensor([[[ 0.2311,    -inf,  0.5954],
         [-0.0703,    -inf,  0.4617],
         [   -inf, -0.4893, -0.9320],
         [ 1.0221, -0.2265,    -inf]],

        [[ 0.2978,    -inf,  2.8054],
         [ 0.8387,    -inf,  0.4485],
         [   -inf,  0.4890,  0.7525],
         [-0.6014,    -inf,  0.3680]]])
tensor([[[0.4099, 0.0000, 0.5901],
         [0.3700, 0.0000, 0.6300],
         [0.0000, 0.6089, 0.3911],
         [0.7771, 0.2229, 0.0000]],

        [[0.0753, 0.0000, 0.9247],
         [0.5963, 0.0000, 0.4037],
         [0.0000, 0.4345, 0.5655],
         [0.2750, 0.0000, 0.7250]]])


In [28]:
batch_size = 2
seq_len = 4
num_experts = 3
embed_size = 8
# (batch_size, seq_len, num_experts)
experts_weights= torch.randn( (batch_size, seq_len, num_experts))
print(experts_weights.unsqueeze(-2).shape)
# 2. get expert_output from experts
 #[num_experts, (batch_size, seq_len, embed_size)]
expert_outputs = [torch.randn( (batch_size, seq_len, embed_size)) for _ in range(num_experts)]
# 3. merge expert_output with router_output
expert_outputs = torch.stack(expert_outputs,dim=-1) # (batch_size, seq_len, embed_size, num_experts)
if torch.isnan(expert_outputs).any():
            expert_outputs[
                torch.isnan(expert_outputs)
            ] = 0
print(expert_outputs.shape)
# Combine expert outputs and gating scores
moe_output = torch.sum(
            experts_weights.unsqueeze(-2) * expert_outputs, dim=-1
        )
print(moe_output.shape)

torch.Size([2, 4, 1, 3])
torch.Size([2, 4, 8, 3])
torch.Size([2, 4, 8])


In [3]:
def fss():
    return 1,2

fss_list = [fss() for _ in range(3)]
out = [x[0] for x in fss_list]
out

[1, 1, 1]

In [5]:
sta_a = "aaa"
sta_b = "bb"
slice1 = slice(0, len(sta_a))
slice2 = slice(slice1.stop, len(sta_a) + len(sta_b))
print(slice1, slice2)
str_ab = sta_a + sta_b
print(str_ab[slice1], str_ab[slice2])

slice(0, 3, None) slice(3, 5, None)
aaa bb
